<a href="https://colab.research.google.com/github/adelacvg/ttts/blob/v4/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b v4 https://github.com/adelacvg/ttts.git

Cloning into 'ttts'...
remote: Enumerating objects: 1126, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 1126 (delta 153), reused 191 (delta 134), pack-reused 892
Receiving objects: 100% (1126/1126), 84.19 MiB | 22.38 MiB/s, done.
Resolving deltas: 100% (650/650), done.


In [2]:
%cd ttts
!pip install -e .

/content/ttts
Obtaining file:///content/ttts
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ttts


In [3]:
!git lfs install
!git clone https://huggingface.co/adelacvg/TTTS_v4

Updated git hooks.
Git LFS initialized.
Cloning into 'TTTS_v4'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 20 (delta 5), reused 0 (delta 0), pack-reused 12 (from 1)
Unpacking objects: 100% (20/20), 6.95 KiB | 1.74 MiB/s, done.
Filtering content: 100% (2/2), 2.33 GiB | 56.00 MiB/s, done.


In [12]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pypinyin einops omegaconf==2.0.6 faiss-cpu==1.8.0 cutlet hangul_romanize unidic
!python -m unidic download

  Preparing metadata (setup.py) ... done
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7406 sha256=11fdcb36697339d8f1412b4998f57bf4b41f5ee7484e7b2f3cef32b22a7c92d6
  Stored in directory: /root/.cache/pip/wheels/7a/72/72/1f3d654c345ea69d5d51b531c90daf7ba14cc555eaf2c64ab0
Successfully built unidic
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:33<00:00, 15.7MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.10/dist-packages/unidic/dicdir


In [13]:
from pypinyin import lazy_pinyin, Style
import torch
import numpy as np
from ttts.utils import vc_utils
import cutlet
from hangul_romanize import Transliter
from hangul_romanize.rule import academic
import torchaudio
from pypinyin import pinyin, lazy_pinyin, Style

device = 'cuda:0'
cond_audio = 'ttts/6.wav'
lang = 'ZH'
text = "大家好，今天来点大家想看的东西。"
# cond_audio = 'ttts/jp.wav'
# lang = 'JP'
# text = '皆さん、こんにちは！今日は皆さんに読んでいただきたいことがあります。'
# cond_audio = 'ttts/en.wav'
# lang = 'EN'
# text = "Hello everyone, here's something you'll want to read today."
# cond_audio = 'ttts/kr.wav'
# lang = 'KR'
# text = "안녕하세요, 여러분, 오늘 읽어보시면 좋을 내용이 있습니다."

katsu = cutlet.Cutlet()
r = Transliter(academic)
def convert_to_latin(text, lang):
    if lang == "ZH":
        text = ' '.join(lazy_pinyin(text, style=Style.TONE3, neutral_tone_with_five=True))
        text = ' '+text+' '
    elif lang == "JP":
        text = katsu.romaji(text)
        text = ' '+text+' '
    elif lang == "EN":
        text = text
        text = ' '+text+' '
    elif lang == "KR":
        text = r.translit(text)
        text = ' '+text+' '
    else:
        return None
    return text

MODELS = {
    'vqvae.pth':'TTTS_v4/model-555.pt',
}
from ttts.gpt.voice_tokenizer import VoiceBpeTokenizer
import torch.nn.functional as F

tok_zh = VoiceBpeTokenizer('ttts/tokenizers/zh_tokenizer.json')
tok_en = VoiceBpeTokenizer('ttts/tokenizers/en_tokenizer.json')
tok_jp = VoiceBpeTokenizer('ttts/tokenizers/jp_tokenizer.json')
tok_kr = VoiceBpeTokenizer('ttts/tokenizers/kr_tokenizer.json')
text = convert_to_latin(text,lang)
if lang == "ZH":
    text = tok_zh.encode(text.lower())
elif lang == "JP":
    text = tok_jp.encode(text.lower())
elif lang == "EN":
    text = tok_en.encode(text.lower())
elif lang == "KR":
    text = tok_kr.encode(text.lower())
else:
    print('not supprt lang')
text = torch.LongTensor(text).unsqueeze(0)
if lang == "ZH":
    text = text + 256*0
    lang = 0
elif lang == "JP":
    text = text + 256*1
    lang = 1
elif lang == "EN":
    text = text + 256*2
    lang = 2
elif lang == "KR":
    text = text + 256*3
    lang = 3

text_tokens = text.to(device)
text_length = torch.LongTensor([text_tokens.shape[1]]).to(device)

from ttts.utils.infer_utils import load_model
import torchaudio
import json
import torchaudio.functional as F
from ttts.utils.data_utils import spectrogram_torch,HParams
hps = HParams(**json.load(open('ttts/vqvae/config_v3.json')))
wav, sr = torchaudio.load(cond_audio)
if wav.shape[0] > 1:
    wav = wav[0].unsqueeze(0)
wav = wav.to(device)
wav32k = F.resample(wav, sr, 32000)
wav32k = wav32k[:,:int(hps.data.hop_length * (wav32k.shape[-1]//hps.data.hop_length))]
wav = torch.clamp(wav32k, min=-1.0, max=1.0)
wav_length = torch.LongTensor([wav.shape[1]])
spec = spectrogram_torch(wav, hps.data.filter_length,
                    hps.data.hop_length, hps.data.win_length, center=False).squeeze(0)
spec_length = torch.LongTensor([
    x//hps.data.hop_length for x in wav_length]).to(device)
vqvae = load_model('vqvae', MODELS['vqvae.pth'], 'ttts/vqvae/config_v3.json', device)
with torch.no_grad():
    wav_out = vqvae.infer(text_tokens, text_length, spec.unsqueeze(0), spec_length)
torchaudio.save('gen.wav', wav_out.squeeze(0).cpu(), 32000)

In [14]:
from IPython.display import Audio
wav = wav_out.detach().cpu()
Audio(wav[0],rate=32000)